In [251]:
from numba import jit, autojit

In [1]:
from __future__ import division
from radd.build import rangl_data
import time
import numpy as np
from lmfit import Parameters, minimize, fit_report
from numpy.random import random_sample as rs
from scipy.stats.mstats import mquantiles as mq
import line_profiler
import memory_profiler

def load_ipython_extension(ip):
      ip.define_magic('lprun', line_profiler.magic_lprun)

def load_ipython_extension(ip):
      ip.define_magic('memit', memory_profiler.magic_memit)
      ip.define_magic('mprun', memory_profiler.magic_mprun)

In [280]:
inits = {k:v for k,v in redf_store['rebsl_boot_popt'].items()}
cinits = {k:v for k,v in redf_store['rebsl_boot_popt'].items()}
theta = {k:v for k,v in redf_store['rebsl_boot_popt'].items()}

vval = cinits.pop('v')
for i in range(2):
      cinits['v'+str(i)] = vval
      
data = redf_store['redata'].copy()
y = data.groupby('Cond').apply(rangl_data, kind='reactive')

In [395]:
a, tr, v, ssv, z  = theta['a'], theta['tr'], theta['v'], -abs(theta['ssv']),  theta['z']

In [396]:
tr=np.array([tr*1.05, tr*.95])

In [397]:
nssd=len(ssd)
dx=np.sqrt(si*dt)
Pg = 0.5*(1 + v*dx/si)
Tg = np.asarray(np.ceil((tb-tr)/dt).astype(int))

Ps = 0.5*(1 + ssv*dx/si)
Ts = np.asarray(np.ceil((tb-ssd)/dt).astype(int))

In [398]:
Tg

array([660, 721])

In [294]:
DVg = z + np.cumsum(np.where((rs((1, ntot, Tg.max())).T < Pg), dx, -dx).T, axis=2)

In [364]:
# Onset Time Bias: ALL CONDITIONS, ALL SSD

DVg = z + np.cumsum(np.where((rs((ncond, ntot, Tg.max())).T < Pg), dx, -dx).T, axis=2)
if bias=='tr':
      init_ss = np.array([[DVc[:nss, ix] for ix in np.where(Ts<Tg[i], Tg[i]-Ts, 0)] for i, DVc in enumerate(DVg)])
DVs = init_ss[:, :, :, None] + np.cumsum(np.where(rs((nss, Ts.max()))<Ps, dx, -dx), axis=1)


# ALL CONDITIONS, ALL SSD
grt = (tr + (np.where(DVg[:, nss:, :].max(axis=2)>=a, np.argmax(DVg[:, nss:, :]>=a, axis=2)*dt, np.nan).T)).T
ert = (tr + (np.where(DVg[:, :nss, :].max(axis=2)>=a, np.argmax(DVg[:, :nss, :]>=a, axis=2)*dt, np.nan).T)).T
ssrt = np.where(np.any(DVs<=0, axis=3), ssd[:, None]+np.argmax(DVs<=0, axis=3)*dt, np.nan)


#collapse across SSD and get average ssrt vector for each condition
c_ssrt = ssrt.mean(axis=1)

# compute RT quantiles for correct and error resp.
gq = np.vstack([mq(rtc[rtc<tb], prob=p)*10 for rtc in grt])
eq = np.array([mq(np.extract(ert[i]<c_ssrt[i], ert[i]), prob=p)*10 for i in range(ncond)])

# Get response and stop accuracy information
gac = np.where(grt<tb, 1, 0).mean(axis=1)
sacc = np.array([1 - np.where(ert[i]<ssrt[i], 1, 0).mean(axis=1) for i in range(ncond)])

yhat = [gac, sacc, gq, eq]
yhat_grouped = np.array([np.hstack([i[ii] for i in yhat]) for ii in range(ncond)])

In [366]:
DVg = z + np.cumsum(np.where((rs((ncond, ntot, Tg.max())).T < Pg), dx, -dx).T, axis=2)
init_ss = np.array([[DVc[:nss, ix] for ix in np.where(Ts<Tg[i], Tg[i]-Ts, 0)] for i, DVc in enumerate(DVg)])
DVs = init_ss[:, :, :, None] + np.cumsum(np.where(rs((nss, Ts.max()))<Ps, dx, -dx), axis=1)


# ALL CONDITIONS, ALL SSD
grt = (tr + (np.where(DVg[:, nss:, :].max(axis=2)>=a, np.argmax(DVg[:, nss:, :]>=a, axis=2)*dt, np.nan).T)).T
ert = (tr + (np.where(DVg[:, :nss, :].max(axis=2)>=a, np.argmax(DVg[:, :nss, :]>=a, axis=2)*dt, np.nan).T)).T
ssrt = np.where(np.any(DVs<=0, axis=3), ssd[:, None]+np.argmax(DVs<=0, axis=3)*dt, np.nan)


array([ 0.32011423,  0.28962716])

In [365]:
yhat_grouped

array([[ 0.946     ,  0.9988    ,  0.9948    ,  0.9672    ,  0.69      ,
         0.178     ,  5.24614233,  5.51614233,  5.72614233,  5.93614233,
         6.20694233,  4.94254233,  5.14614233,  5.31614233,  5.46614233,
         5.71794233],
       [ 0.9796    ,  0.9968    ,  0.9808    ,  0.868     ,  0.304     ,
         0.0668    ,  4.94127163,  5.22127163,  5.42127163,  5.66127163,
         6.02127163,  4.73847163,  4.93127163,  5.09127163,  5.25127163,
         5.50127163]])

2445

In [360]:
np.sum(x)

2445

In [391]:
# ALL CONDITIONS, ALL SSD
grt = (tr + (np.where(DVg[:, nss:, :].max(axis=2)>=a, np.argmax(DVg[:, nss:, :]>=a, axis=2)*dt, np.nan).T)).T
ert = (tr + (np.where(DVg[:, :nss, :].max(axis=2)>=a, np.argmax(DVg[:, :nss, :]>=a, axis=2)*dt, np.nan).T)).T
ssrt = np.where(np.any(DVs<=0, axis=3), ssd[:, None]+np.argmax(DVs<=0, axis=3)*dt, np.nan)

In [329]:
grt

array([[ 0.52661423,  0.50361423,  0.62361423, ...,  0.52361423,
         0.54261423,  0.62461423],
       [ 0.49612716,  0.52012716,  0.57012716, ...,  0.50912716,
         0.55112716,  0.50712716]])

In [380]:
#collapse across SSD and get average ssrt vector for each condition
c_ssrt = ssrt.mean(axis=1)

# compute RT quantiles for correct and error resp.
gq = np.vstack([mq(rtc[rtc<tb], prob=p)*10 for rtc in grt])
eq = np.array([mq(np.extract(ert[i]<c_ssrt[i], ert[i]), prob=p)*10 for i in range(ncond)])

# Get response and stop accuracy information
gac = np.where(grt<tb, 1, 0).mean(axis=1)
sacc = np.array([1 - np.where(ert[i]<ssrt[i], 1, 0).mean(axis=1) for i in range(ncond)])

yhat = [gac, sacc, gq, eq]
yhat_grouped = np.array([np.hstack([i[ii] for i in yhat]) for ii in range(ncond)])

SyntaxError: 'return' outside function (<ipython-input-380-4bfd482ab8cb>, line 15)

In [276]:
%%timeit
iter_recost_new2(theta, y, nruns=5, ntrials=5000)
#iter_recost(cinits, y, nruns=5, ntrials=5000)

1 loops, best of 3: 3.26 s per loop


In [368]:
a=np.array(theta['a']*1.05, theta['a']*.9)

In [389]:
grt = np.where(DVg[:, nss:, :].max(axis=2)>=a, tr + np.argmax(DVg[:, nss:, :]>=a, axis=2)*dt, np.nan)
ert = np.where(DVg[:, :nss, :].max(axis=2)>=a, tr + np.argmax(DVg[:, :nss, :]>=a, axis=2)*dt, np.nan)
ssrt = np.where(np.any(DVs<=0, axis=3), ssd[:, None]+np.argmax(DVs<=0, axis=3)*dt, np.nan)


In [392]:
grt

array([[ 0.62661423,  0.65861423,  0.57961423, ...,  0.61761423,
         0.58661423,  0.64261423],
       [ 0.54961423,  0.55861423,  0.57661423, ...,  0.56061423,
         0.61061423,  0.56161423]])

In [275]:
def iter_recost_new2(inits, y, nruns=5, ntrials=5000):
      ip=inits.copy()
      for i in range(nruns):
            cost = recost_new2(ip, y, ntrials=ntrials)

def recost_new2(theta, y, bias=['v'], ncond=2, ntrials=2000, pGo=.5, wts=None, nssd=5, ssd=np.arange(.2, .45, .05), p=np.array([.1, .3, .5, .7, .9])):

      if not type(theta)==dict:
            theta = theta.valuesdict()

      for bk in bias:
            theta[bk] = np.array([theta[bk+str(i)] for i in range(ncond)])

      yhat = simulate_full_new2(theta['a'], theta['tr'], theta['v'], theta['ssv'], theta['z'], ncond=ncond, ssd=ssd, nss=int((1-pGo)*ntrials), nssd=len(ssd), ntot=ntrials)
      y=np.vstack(y)
      cost = np.vstack(y) - yhat
      return cost


def simulate_full_new2(a, tr, v, ssv, z, analyze=True, ncond=2, p=np.array([.1, .3, .5, .7, .9]), ssd=np.arange(.2, .45, .05), nssd=5, ntot=2000, tb=0.650, dt=.0005, si=.01, nss=1000):
      """

      Simulates all Conditions, SSD, trials, timepoints simultaneously by
      treating drift-rate as a vector, containing a value for each condition.

      args:
            a, tr, ssv, z (float):        model parameters (excl 'v')
            v (array):                    array of drift-rates (1/cond)
            ssd  (array):                 full set of stop signal delays
            nss  (int):                   number of stop trials
            ntot (int):                   number of total trials
            tb (float):                   time boundary

      returns:

            DVg (Go Process):       3d array for all conditions, trials, timepoints
                                    (i.e. DVg = [COND [NTrials [NTime]]] )
                                    All conditions are simulated simultaneously (i.e., BSL & PNL)

            DVs (Stop Process):     4d array for all conditions, SSD, SS trials, timepoints.
                                    i.e. ( DVs = [COND [SSD [nSSTrials [NTime]]]] )
                                    All ss decision traces are initiated from DVg[Cond](t=SSD | SSD<tr)

      Output can be passed to <analyze_reactive_full()> to extract
      expected values to be entered into the cost f(x)
      """

      #nssd=len(ssd)
      #nss=int(ntot/2)
      dx=np.sqrt(si*dt)
      Pg = 0.5*(1 + v*dx/si)
      Tg = np.ceil((tb-tr)/dt).astype(int)

      Ps = 0.5*(1 + ssv*dx/si)
      Ts = np.ceil((tb-ssd)/dt).astype(int)

      # ALL CONDITIONS, ALL SSD
      DVg = z + np.cumsum(np.where((rs((ncond, ntot, Tg)).T < Pg), dx, -dx).T, axis=2)
      init_ss = np.array([[DVc[:nss, ix] for ix in np.where(Ts<Tg, Tg-Ts, 0)] for DVc in DVg])
      DVs = init_ss[:, :, :, None] + np.cumsum(np.where(rs((nss, Ts.max()))<Ps, dx, -dx), axis=1) 


      # ALL CONDITIONS, ALL SSD
      grt = np.where(DVg[:, nss:, :].max(axis=2)>=a, tr + np.argmax(DVg[:, nss:, :]>=a, axis=2)*dt, np.nan)
      ert = np.where(DVg[:, :nss, :].max(axis=2)>=a, tr + np.argmax(DVg[:, :nss, :]>=a, axis=2)*dt, np.nan)
      ssrt = np.where(np.any(DVs<=0, axis=3), ssd[:, None]+np.argmax(DVs<=0, axis=3)*dt, np.nan)

      #collapse across SSD and get average ssrt vector for each condition
      c_ssrt = ssrt.mean(axis=1)

      # compute RT quantiles for correct and error resp.
      gq = np.vstack([mq(rtc[rtc<tb], prob=p)*10 for rtc in grt])
      eq = np.array([mq(np.extract(ert[i]<c_ssrt[i], ert[i]), prob=p)*10 for i in range(ncond)])

      # Get response and stop accuracy information
      gac = np.where(grt<tb, 1, 0).mean(axis=1)
      sacc = np.array([1 - np.where(ert[i]<ssrt[i], 1, 0).mean(axis=1) for i in range(ncond)])

      yhat = [gac, sacc, gq, eq]
      yhat_grouped = np.array([np.hstack([i[ii] for i in yhat]) for ii in range(ncond)])

      return yhat_grouped

In [237]:
#3290 function calls in 3.539 seconds
#3290 function calls in 3.355 seconds
def iter_recost_new1(inits, y, nruns=5, ntrials=5000):
      ip=inits.copy()
      for i in range(nruns):
            cost = recost_new1(ip, y, ntrials=ntrials)

def recost_new1(theta, y, bias=['v'], ncond=2, ntrials=2000, wts=None, nssd=5, ssd=np.arange(.2, .45, .05), p=np.array([.1, .3, .5, .7, .9])):

      if not type(theta)==dict:
            theta = theta.valuesdict()

      for bk in bias:
            theta[bk] = np.array([theta[bk+str(i)] for i in range(ncond)])

      yhat = simulate_full_new1(theta['a'], theta['tr'], theta['v'], theta['ssv'], theta['z'], nssd=nssd, ncond=ncond, ssd=ssd, ntot=ntrials)
      y=np.vstack(y)
      cost = np.vstack(y) - yhat
      return cost




def simulate_full_new1(a, tr, v, ssv, z, analyze=True, ncond=2, nssd=5, p=np.array([.1, .3, .5, .7, .9]), ssd=np.arange(.2, .45, .05), ntot=2000, tb=0.650, dt=.0005, si=.01):
      """

      Simulates all Conditions, SSD, trials, timepoints simultaneously by
      treating drift-rate as a vector, containing a value for each condition.

      args:
            a, tr, ssv, z (float):        model parameters (excl 'v')
            v (array):                    array of drift-rates (1/cond)
            ssd  (array):                 full set of stop signal delays
            nss  (int):                   number of stop trials
            ntot (int):                   number of total trials
            tb (float):                   time boundary

      returns:

            DVg (Go Process):       3d array for all conditions, trials, timepoints
                                    (i.e. DVg = [COND [NTrials [NTime]]] )
                                    All conditions are simulated simultaneously (i.e., BSL & PNL)

            DVs (Stop Process):     4d array for all conditions, SSD, SS trials, timepoints.
                                    i.e. ( DVs = [COND [SSD [nSSTrials [NTime]]]] )
                                    All ss decision traces are initiated from DVg[Cond](t=SSD | SSD<tr)

      Output can be passed to <analyze_reactive_full()> to extract
      expected values to be entered into the cost f(x)
      """
      
      nssd=len(ssd)
      nss = int(ntot/nssd)
      
      dx=np.sqrt(si*dt)
      Pg = 0.5*(1 + v*dx/si)
      Tg = np.ceil((tb-tr)/dt).astype(int)

      Ps = 0.5*(1 + ssv*dx/si)
      Ts = np.ceil((tb-ssd)/dt).astype(int)

      # ALL CONDITIONS, ALL SSD
      DVg = z + np.cumsum(np.where((rs((ncond, ntot, Tg)).T < Pg), dx, -dx).T, axis=2)
      init_ss = np.array([[DVc[:nss, ix] for ix in np.where(Ts<Tg, Tg-Ts, 0)] for DVc in DVg])
      DVs = init_ss[:,:,:,None]+np.cumsum(np.where(rs((ntot*ncond, Ts.max()))<Ps, dx, -dx), axis=1).reshape(ncond, nssd, nss, Ts.max())

      # ALL CONDITIONS, ALL SSD
      grt = np.where(DVg[:, nss:, :].max(axis=2)>=a, tr + np.argmax(DVg[:, nss:, :]>=a, axis=2)*dt, np.nan)
      ert = np.where(DVg[:, :nss, :].max(axis=2)>=a, tr + np.argmax(DVg[:, :nss, :]>=a, axis=2)*dt, np.nan)
      ssrt = np.where(np.any(DVs<=0, axis=3), ssd[:, None]+np.argmax(DVs<=0, axis=3)*dt, np.nan)

      #collapse across SSD and get average ssrt vector for each condition
      c_ssrt = ssrt.mean(axis=1)

      # compute RT quantiles for correct and error resp.
      gq = np.vstack([mq(rtc[rtc<tb], prob=p)*10 for rtc in grt])
      eq = np.array([mq(np.extract(ert[i]<c_ssrt[i], ert[i]), prob=p)*10 for i in range(ncond)])

      # Get response and stop accuracy information
      gac = np.where(grt<tb, 1, 0).mean(axis=1)
      sacc = np.array([1 - np.where(ert[i]<ssrt[i], 1, 0).mean(axis=1) for i in range(ncond)])

      yhat = [gac, sacc, gq, eq]
      yhat_grouped = np.array([np.hstack([i[ii] for i in yhat]) for ii in range(ncond)])

      return yhat_grouped

      #else:

      #     return DVg, DVs

In [271]:

def iter_recost(inits, y, nruns=5, ntrials=5000):
      ip=inits.copy()
      for i in range(nruns):
            cost = recost(ip, y, ntrials=ntrials)


def recost(theta, y, bias=['v'], ncond=2, ntrials=2000, wts=None, pGo=.5, ssd=np.arange(.2, .45, .05), p=np.array([.1, .3, .5, .7, .9])):

      if not type(theta)==dict:
            theta = theta.valuesdict()

      #theta[bias] = np.array([bias+str(i) for i in range(ncond)])
      for bk in bias:
            theta[bk] = np.array([theta[bk+str(i)] for i in range(ncond)])
      a, tr, v, ssv, z  = theta['a'], theta['tr'], theta['v'], -abs(theta['ssv']),  theta['z']
      nss = int((1-pGo)*ntrials)

      yhat = simulate_full(a, tr, v, ssv, z, p=p, ncond=ncond, ssd=ssd, nss=nss, ntot=ntrials)
      #wtc, wte = wts[0], wts[1]
      y=np.vstack(y)
      cost = np.vstack(y) - yhat
      #cost = np.hstack([y[:6]-yhat[:6], wtc*y[6:11]-wtc*yhat[6:11], wte*y[11:]-wte*yhat[11:]]).astype(np.float32)

      return cost



def set_bounds(a=(.01, .6), tr=(.001, .5), v=(.01, 4.), z=(.001, .9), ssv=(-4., -.01)):

      return {'a': a, 'tr': tr, 'v': v, 'ssv': ssv, 'z': z}



def simulate_full(a, tr, v, ssv, z, analyze=True, ncond=1, p=np.array([.1, .3, .5, .7, .9]), ssd=np.arange(.2, .45, .05), nss=1000, ntot=2000, tb=0.650, dt=.0005, si=.01):
      """

      Simulates all Conditions, SSD, trials, timepoints simultaneously by
      treating drift-rate as a vector, containing a value for each condition.

      args:
            a, tr, ssv, z (float):        model parameters (excl 'v')
            v (array):                    array of drift-rates (1/cond)
            ssd  (array):                 full set of stop signal delays
            nss  (int):                   number of stop trials
            ntot (int):                   number of total trials
            tb (float):                   time boundary

      returns:

            DVg (Go Process):       3d array for all conditions, trials, timepoints
                                    (i.e. DVg = [COND [NTrials [NTime]]] )
                                    All conditions are simulated simultaneously (i.e., BSL & PNL)

            DVs (Stop Process):     4d array for all conditions, SSD, SS trials, timepoints.
                                    i.e. ( DVs = [COND [SSD [nSSTrials [NTime]]]] )
                                    All ss decision traces are initiated from DVg[Cond](t=SSD | SSD<tr)

      Output can be passed to <analyze_reactive_full()> to extract
      expected values to be entered into the cost f(x)
      """

      nssd=len(ssd)
      dx=np.sqrt(si*dt)
      Pg = 0.5*(1 + v*dx/si)
      Tg = np.ceil((tb-tr)/dt).astype(int)

      Ps = 0.5*(1 + ssv*dx/si)
      Ts = np.ceil((tb-ssd)/dt).astype(int)

      # ALL CONDITIONS, ALL SSD
      DVg = z + np.cumsum(np.where((rs((ncond, ntot, Tg)).T < Pg), dx, -dx).T, axis=2)
      init_ss = np.array([np.array([DVc[:nss, ix] for ix in np.where(Ts<Tg, Tg-Ts, 0)]) for DVc in DVg])
      DVs = init_ss[:, :, :, None] + np.cumsum(np.where(rs((ncond, nssd, nss, Ts.max()))<Ps, dx, -dx), axis=3)

      # ALL CONDITIONS, ALL SSD
      grt = np.where(DVg[:, nss:, :].max(axis=2)>=a, tr + np.argmax(DVg[:, nss:, :]>=a, axis=2)*dt, np.nan)
      ert = np.where(DVg[:, :nss, :].max(axis=2)>=a, tr + np.argmax(DVg[:, :nss, :]>=a, axis=2)*dt, np.nan)
      ssrt = np.where(np.any(DVs<=0, axis=3), ssd[:, None]+np.argmax(DVs<=0, axis=3)*dt, np.nan)

      #collapse across SSD and get average ssrt vector for each condition
      c_ssrt = ssrt.mean(axis=1)

      # compute RT quantiles for correct and error resp.
      gq = np.vstack([mq(rtc[rtc<tb], prob=p)*10 for rtc in grt])
      eq = np.array([mq(np.extract(ert[i]<c_ssrt[i], ert[i]), prob=p)*10 for i in range(ncond)])

      # Get response and stop accuracy information
      gac = np.where(grt<tb, 1, 0).mean(axis=1)
      sacc = np.array([1 - np.where(ert[i]<ssrt[i], 1, 0).mean(axis=1) for i in range(ncond)])

      yhat = [gac, sacc, gq, eq]
      yhat_grouped = np.array([np.hstack([i[ii] for i in yhat]) for ii in range(ncond)])

      return yhat_grouped

In [268]:
theta['v'] = np.array([theta['v']*1.05, theta['v']*.99])
v = theta['v']
theta = {k:v for k,v in redf_store['rebsl_boot_popt'].items()}
theta['v'] = np.array([theta['v']*1.05, theta['v']*.99])
v = theta['v']


In [287]:
%%timeit
a=theta['a']
tr=theta['tr']
ssv=-abs(theta['ssv'])
v = theta['v']
z=theta['z']
ncond=2
p=np.array([.1, .3, .5, .7, .9])
ssd=np.arange(.2, .45, .05)
ntot=5000
nssd=len(ssd)
nss=int(ntot/2/len(ssd))
tb=0.650
dt=.0005
si=.01

dx=np.sqrt(si*dt)
Pg = 0.5*(1 + v*dx/si)
Tg = np.ceil((tb-tr)/dt).astype(int)

Ps = 0.5*(1 + ssv*dx/si)
Ts = np.ceil((tb-ssd)/dt).astype(int)

# ALL CONDITIONS, ALL SSD
DVg = z + np.cumsum(np.where((rs((ncond, ntot, Tg)).T < Pg), dx, -dx).T, axis=2)
#init_ss = np.array([np.array([DVc[:nss, ix] for ix in np.where(Ts<Tg, Tg-Ts, 0)]) for DVc in DVg])
#DVs = init_ss[:, :, :, None] + np.cumsum(np.where(rs((ncond, nssd, nss, Ts.max()))<Ps, dx, -dx), axis=3)
init_ss = np.array([[DVc[:nss, ix] for ix in np.where(Ts<Tg, Tg-Ts, 0)] for DVc in DVg])
DVs = init_ss[:, :, :, None] + np.cumsum(np.where(rs((nss*ncond, Ts.max()))<Ps, dx, -dx), axis=1).reshape(ncond, nssd, nss, Ts.max())
#if analyze:

# ALL CONDITIONS, ALL SSD
grt = np.where(DVg[:, nss:, :].max(axis=2)>=a, tr + np.argmax(DVg[:, nss:, :]>=a, axis=2)*dt, np.nan)
ert = np.where(DVg[:, :nss, :].max(axis=2)>=a, tr + np.argmax(DVg[:, :nss, :]>=a, axis=2)*dt, np.nan)
ssrt = np.where(np.any(DVs<=0, axis=3), ssd[:, None]+np.argmax(DVs<=0, axis=3)*dt, np.nan)

ValueError: total size of new array must be unchanged

In [ ]:
init_ss.shape[:, :, ]

In [579]:
ssrt2 = np.where(np.any(DVs2<=0, axis=3), ssd[:, None]+np.argmax(DVs2<=0, axis=3)*dt, np.nan)


In [593]:
init_ss = np.array([np.array([DVc[:nss, ix] for ix in np.where(Ts<Tg, Tg-Ts, 0)]) for DVc in DVg])
DVs1 = init_ss[:, :, :, None] + np.cumsum(np.where(rs((ncond, nssd, nss, Ts.max()))<Ps, dx, -dx), axis=2)
DVs2 = init_ss[:, :, :, None] + b#np.cumsum(np.where(rs((nss, Ts.max()))<Ps, dx, -dx), axis=1) 

In [590]:
a = np.cumsum(np.where(rs((nss, Ts.max()))<Ps, dx, -dx), axis=1) 
b=np.copy(a)

In [615]:
a = init_ss[:, :, :, None] + np.array([np.cumsum(np.where(rs((nss, Ts.max()))<Ps, dx, -dx), axis=1)]*5)

In [616]:
a.shape

(2, 5, 2500, 900)

In [591]:
b.shape

(5, 2500, 900)

In [52]:
x = init_ss[:,:,:]

In [57]:
x.T + n

ValueError: operands could not be broadcast together with shapes (2500,5,2) (2,1,5,2500,900) 

In [63]:
dvs.shape
#init_ss.shape

(2500, 900)

In [64]:
sxi = np.array([np.array([DVc[:nss, ix] for ix in np.where(Ts<Tg, Tg-Ts, 0)]) for DVc in DVg])

In [90]:
x = sxi[:, :, :nss, None] + dvs
x[1,1,1,0]

0.15871869060699953

In [75]:
(sxi[:, :, :nss, None] + dvs).shape

(2, 5, 2500, 900)

In [69]:

sxi.shape.reshape()
dvs.shape


(2500, 900)

In [60]:
dvs = np.cumsum(np.where(rs((nss, Ts.max()))<Ps, dx, -dx), axis=1)
np.sum(init_ss, dvs)

TypeError: only length-1 arrays can be converted to Python scalars

In [93]:

a=theta['a']
tr=theta['tr']
ssv=-abs(theta['ssv'])
v = theta['v']
z=theta['z']
ncond=2
p=np.array([.1, .3, .5, .7, .9])
ssd=np.arange(.2, .45, .05)
ntot=5000
nssd=len(ssd)
nss=int(ntot/len(ssd))
nss=5000/2
tb=0.650
dt=.0005
si=.01

dx=np.sqrt(si*dt)
Pg = 0.5*(1 + v*dx/si)
Tg = np.ceil((tb-tr)/dt).astype(int)

Ps = 0.5*(1 + ssv*dx/si)
Ts = np.ceil((tb-ssd)/dt).astype(int)

# ALL CONDITIONS, ALL SSD
DVg = z + np.cumsum(np.where((rs((ncond, ntot, Tg)).T < Pg), dx, -dx).T, axis=2)
init_ss = np.array([np.array([DVc[:nss, ix] for ix in np.where(Ts<Tg, Tg-Ts, 0)]) for DVc in DVg])
DVs = init_ss[:, :, :, None] + np.cumsum(np.where(rs((nss, Ts.max()))<Ps, dx, -dx), axis=1) 
#DVs2 = init_ss[:, :, :, None] + np.array([np.cumsum(np.where(rs((nssd, nss, Ts.max()))<Ps, dx, -dx), axis=1)])
#if analyze:

# ALL CONDITIONS, ALL SSD
grt = np.where(DVg[:, nss:, :].max(axis=2)>=a, tr + np.argmax(DVg[:, nss:, :]>=a, axis=2)*dt, np.nan)
ert = np.where(DVg[:, :nss, :].max(axis=2)>=a, tr + np.argmax(DVg[:, :nss, :]>=a, axis=2)*dt, np.nan)
ssrt1 = np.where(np.any(DVs1<=0, axis=3), ssd[:, None]+np.argmax(DVs1<=0, axis=3)*dt, np.nan)
ssrt2 = np.where(np.any(DVs2<=0, axis=3), ssd[:, None]+np.argmax(DVs2<=0, axis=3)*dt, np.nan)

#collapse across SSD and get average ssrt vector for each condition
c_ssrt1 = np.nanmean(ssrt1, axis=1)
c_ssrt2 = np.nanmean(ssrt2, axis=1)

# compute RT quantiles for correct and error resp.
gq = np.vstack([mq(rtc[rtc<tb], prob=p)*10 for rtc in grt])
eq1 = np.array([mq(np.extract(ert[i]<c_ssrt1[i], ert[i]), prob=p)*10 for i in range(ncond)])
eq2 = np.array([mq(np.extract(ert[i]<c_ssrt2[i], ert[i]), prob=p)*10 for i in range(ncond)])

# Get response and stop accuracy information
gac = np.where(grt<tb, 1, 0).mean(axis=1)
sacc1 = np.array([1 - np.where(ert[i]<ssrt1[i], 1, 0).mean(axis=1) for i in range(ncond)])
sacc2 = np.array([1 - np.where(ert[i]<ssrt2[i], 1, 0).mean(axis=1) for i in range(ncond)])

yhat1 = [gac, sacc1, gq, eq1]
yhat2 = [gac, sacc2, gq, eq2]

yhat_grouped1 = np.array([np.hstack([i[ii] for i in yhat1]) for ii in range(ncond)])
yhat_grouped2 = np.array([np.hstack([i[ii] for i in yhat2]) for ii in range(ncond)])



In [226]:

def simulate_full(a, tr, v, ssv, z, analyze=True, ncond=1, p=np.array([.1, .3, .5, .7, .9]), ssd=np.arange(.2, .45, .05), ntot=2000, tb=0.650, dt=.0005, si=.01):
      """

      Simulates all Conditions, SSD, trials, timepoints simultaneously by
      treating drift-rate as a vector, containing a value for each condition.

      args:
            a, tr, ssv, z (float):        model parameters (excl 'v')
            v (array):                    array of drift-rates (1/cond)
            ssd  (array):                 full set of stop signal delays
            nss  (int):                   number of stop trials
            ntot (int):                   number of total trials
            tb (float):                   time boundary

      returns:

            DVg (Go Process):       3d array for all conditions, trials, timepoints
                                    (i.e. DVg = [COND [NTrials [NTime]]] )
                                    All conditions are simulated simultaneously (i.e., BSL & PNL)

            DVs (Stop Process):     4d array for all conditions, SSD, SS trials, timepoints.
                                    i.e. ( DVs = [COND [SSD [nSSTrials [NTime]]]] )
                                    All ss decision traces are initiated from DVg[Cond](t=SSD | SSD<tr)

      Output can be passed to <analyze_reactive_full()> to extract
      expected values to be entered into the cost f(x)
      """

      nssd=len(ssd)
      dx=np.sqrt(si*dt)
      Pg = 0.5*(1 + v*dx/si)
      Tg = np.ceil((tb-tr)/dt).astype(int)

      Ps = 0.5*(1 + ssv*dx/si)
      Ts = np.ceil((tb-ssd)/dt).astype(int)

      # ALL CONDITIONS, ALL SSD
      DVg = z + np.cumsum(np.where((rs((ncond, ntot, Tg)).T < Pg), dx, -dx).T, axis=2)
      init_ss = np.array([[DVc[:nss, ix] for ix in np.where(Ts<Tg, Tg-Ts, 0)] for DVc in DVg])
      DVs = init_ss[:, :, :, None] + np.cumsum(np.where(rs((nss, Ts.max()))<Ps, dx, -dx), axis=1) 


      # ALL CONDITIONS, ALL SSD
      grt = np.where(DVg[:, nss:, :].max(axis=2)>=a, tr + np.argmax(DVg[:, nss:, :]>=a, axis=2)*dt, np.nan)
      ert = np.where(DVg[:, :nss, :].max(axis=2)>=a, tr + np.argmax(DVg[:, :nss, :]>=a, axis=2)*dt, np.nan)
      ssrt = np.where(np.any(DVs<=0, axis=3), ssd[:, None]+np.argmax(DVs<=0, axis=3)*dt, np.nan)

      #collapse across SSD and get average ssrt vector for each condition
      c_ssrt = ssrt.mean(axis=1)

      # compute RT quantiles for correct and error resp.
      gq = np.vstack([mq(rtc[rtc<tb], prob=p)*10 for rtc in grt])
      eq = np.array([mq(np.extract(ert[i]<c_ssrt[i], ert[i]), prob=p)*10 for i in range(ncond)])

      # Get response and stop accuracy information
      gac = np.where(grt<tb, 1, 0).mean(axis=1)
      sacc = np.array([1 - np.where(ert[i]<ssrt[i], 1, 0).mean(axis=1) for i in range(ncond)])

      yhat = [gac, sacc, gq, eq]
      yhat_grouped = np.array([np.hstack([i[ii] for i in yhat]) for ii in range(ncond)])

      return yhat_grouped

In [139]:
c_ssrt1 = np.nanmean(ssrt1, axis=1)

2500

In [156]:
y[1]

array([ 0.92724562,  0.9934426 ,  0.98524588,  0.92540985,  0.5942623 ,
        0.18114755,  5.18260765,  5.45125008,  5.71676922,  5.97715616,
        6.25050116,  5.04096556,  5.31758499,  5.45331287,  5.71764088,
        5.98438311], dtype=float32)

In [161]:
yhat_grouped2

array([[ 0.9628    ,  0.9988    ,  0.9884    ,  0.9436    ,  0.484     ,
         0.106     ,  5.08370698,  5.36370698,  5.55370698,  5.78370698,
         6.11370698,  4.88370698,  5.05370698,  5.19370698,  5.34310698,
         5.56750698],
       [ 0.9628    ,  0.9988    ,  0.988     ,  0.944     ,  0.4704    ,
         0.0936    ,  5.10370698,  5.37370698,  5.59370698,  5.80370698,
         6.14370698,  4.83370698,  5.03370698,  5.16370698,  5.34370698,
         5.59370698]])

In [142]:
eq1 = np.array([mq(ert[0][ert[0]<c_ssrt1[0]], prob=p)*10 for i in range(ncond)])
eq2 = np.array([mq(ert[0][ert[0]<c_ssrt2[0]], prob=p)*10 for i in range(ncond)])

In [143]:
eq1

array([[ 4.79690698,  4.92530698,  5.03370698,  5.12370698,  5.26050698],
       [ 4.79690698,  4.92530698,  5.03370698,  5.12370698,  5.26050698]])

In [144]:
eq2

array([[ 4.88370698,  5.05370698,  5.19370698,  5.34310698,  5.56750698],
       [ 4.88370698,  5.05370698,  5.19370698,  5.34310698,  5.56750698]])

In [121]:
print DVs1[0,0,6,899]
print DVs2[0,0,6,899]


-0.2281210695
-0.187871845905


In [168]:
ssrt2[1][0]

array([ 0.3465,  0.3235,  0.3365, ...,  0.4075,  0.3265,  0.4725])

In [225]:
#sns.distplot(c_ssrt1[1])
sns.kdeplot(c_ssrt2[1])
sns.kdeplot(grt[1])

#sns.distplot(ert[1])

In [214]:
data.query('trial_type==stop & acc=0').rt.values

SyntaxError: can't assign to operator (<unknown>, line 1)

In [219]:
ert_data = data.query('trial_type=="stop" & acc==0').rt.values
grt_data = data.query('trial_type=="go" & acc==1').rt.values
sns.kdeplot(ert_data, shade=True, label='err data')
sns.kdeplot(grt_data, shade=True, label='cor data')
plt.gca().set_xlim(.4,.7)

(0.4, 0.7)

In [211]:
ert_bsl = ert[0]
srt_bsl1 = c_ssrt1[0]
srt_bsl2 = c_ssrt2[0]
sns.kdeplot(ert_bsl, shade=True)

sns.kdeplot(ert_bsl[ert_bsl<srt_bsl1], shade=True, label='err rt 1')
sns.kdeplot(ert_bsl[ert_bsl<srt_bsl], shade=True, label='err rt 2')

In [196]:
sns.distplot(c_ssrt1[1])
sns.distplot(c_ssrt2[1])
sns.kdeplot(grt[1], shade=True)
e300 = ert[1][ert[1]<.4]

In [198]:
sns.kdeplot(e300, shade=True)

ZeroDivisionError: 0.0 cannot be raised to a negative power

In [91]:
c_ssrt1

array([[ 0.5139,  0.4467,  0.4649, ...,  0.5181,  0.4727,  0.4491],
       [ 0.5357,  0.5281,  0.4837, ...,  0.4775,  0.4815,  0.5189]])

In [92]:
c_ssrt2

array([[ nan,  nan,  nan, ...,  0.3,  0.3,  0.3],
       [ nan,  nan,  nan, ...,  0.3,  0.3,  0.3]])

In [ ]:
gq = np.vstack([mq(rtc[rtc<tb], prob=p)*10 for rtc in grt])
eq1 = mq(np.extract(ert[i]<c_ssrt1[i], ert[i]), prob=p)#*10 for i in range(ncond)])
eq2 = mq(np.extract(ert[i]<c_ssrt2[i], ert[i]), prob=p)#*10 for i in range(ncond)])

In [710]:
c_ssrt1 = np.nanmean(ssrt1, axis=1)
c_ssrt2 = np.nanmean(ssrt2, axis=1)
mq(ert[0][ert[0]<c_ssrt2[0]], prob=p)# ert[i]<c_ssrt1[i], ert[i], ), prob=p)*10

array([ 0.4793707,  0.4983707,  0.5128707,  0.5263707,  0.5483707])

In [707]:
c_ssrt1 = np.mean(ssrt1, axis=1)
c_ssrt2 = np.mean(ssrt2, axis=1)

In [668]:
#np.count_nonzero(ssrt1[1][4])
np.count_nonzero(~np.isnan(ssrt1[0][1]))

2500

In [703]:

#pd.Series(ssrt1[1][5]).hist()
#pd.Series(ssrt2[1][5]).hist()
print mq(ssrt1[0][2], prob=p)
print mq(ssrt2[0][2], prob=p)

[ 0.4145  0.4385  0.4585  0.4815  0.5205]
[ 0.4125  0.4375  0.4575  0.4785  0.5145]


In [32]:
np.nanmean(np.nanmean(ssrt2.mean(axis=1), axis=1))

ERROR! Session/line number was not unique in database. History logging moved to new session 1550


0.49574673387096779

In [31]:
np.nanmean(np.nanmean(ssrt1.mean(axis=1), axis=1))

0.49531124712771768

In [46]:
ssrt1[0][1].max()

0.60650000000000004

In [47]:
ssrt2[0][1].max()

0.57650000000000001

In [37]:
sns.distplot()

AttributeError: max must be larger than min in range parameter.

In [ ]:
plt.hist(np.nanmean(ssrt1, axis=1))

In [552]:
c_ssrt1 = np.nanmean(ssrt1, axis=1)
c_ssrt2 = np.nanmean(ssrt2, axis=1)
mq(c_ssrt1[i], prob=p)*10

array([ 4.661,  4.819,  4.939,  5.059,  5.251])

In [540]:
acc1 = np.array([np.where(ert[i]<ssrt1[i], 1, 0).mean(axis=1).astype(np.float16) for i in range(ncond)])
acc2 = np.array([np.where(ert[i]<ssrt2[i], 1, 0).mean(axis=1) for i in range(ncond)])

In [542]:
1 - acc1

array([[ 0.99853516,  0.98779297,  0.92773438,  0.39697266,  0.06933594],
       [ 0.99902344,  0.9921875 ,  0.95605469,  0.53515625,  0.12939453]], dtype=float16)

In [543]:
1 - acc2

array([[ 0.9972,  0.9844,  0.934 ,  0.4028,  0.0668],
       [ 0.9984,  0.9932,  0.9596,  0.5336,  0.1312]])

In [531]:
print np.nanmean(ssrt2, axis=1).mean(axis=1)
print np.nanmean(ssrt1, axis=1).mean(axis=1)

[ 0.49596162  0.49349324]
[ 0.49717488  0.49484618]


In [621]:
yhat_grouped1

array([[ 0.9868    ,  0.9984    ,  0.9876    ,  0.928     ,  0.3976    ,
         0.0876    ,  5.02370698,  5.27370698,  5.47370698,  5.69370698,
         6.02370698,  4.75370698,  4.92330698,  5.00370698,  5.08370698,
         5.24370698],
       [ 0.9612    ,  0.9996    ,  0.99      ,  0.9452    ,  0.504     ,
         0.112     ,  5.12370698,  5.39370698,  5.60370698,  5.83370698,
         6.12370698,  4.81370698,  4.95430698,  5.07370698,  5.14370698,
         5.28370698]])

In [620]:
yhat_grouped2

array([[ 0.9868    ,  0.9984    ,  0.9876    ,  0.928     ,  0.3976    ,
         0.0876    ,  5.02370698,  5.27370698,  5.47370698,  5.69370698,
         6.02370698,  4.81370698,  5.00370698,  5.13370698,  5.28510698,
         5.54150698],
       [ 0.9612    ,  0.9996    ,  0.99      ,  0.9452    ,  0.504     ,
         0.112     ,  5.12370698,  5.39370698,  5.60370698,  5.83370698,
         6.12370698,  4.89190698,  5.09370698,  5.23370698,  5.37370698,
         5.63370698]])

In [483]:
#init_ss[:, :, 1] + 
DVs1[0, 4, 1][-10:]

array([-0.26166209, -0.26389816, -0.26613423, -0.26389816, -0.26613423,
       -0.26837029, -0.26613423, -0.26837029, -0.27060636, -0.26837029])

In [484]:
DVs2[0, 4, 1][-10:]

array([-0.06041597, -0.0581799 , -0.06041597, -0.06265204, -0.06488811,
       -0.06265204, -0.06488811, -0.06265204, -0.06488811, -0.06712418])

In [393]:
sacc1

array([[ 0.9952,  0.9892,  0.9252,  0.3968,  0.0676],
       [ 0.9996,  0.994 ,  0.9548,  0.4936,  0.1176]])

In [394]:
sacc2

array([[ 0.9968,  0.9892,  0.9376,  0.3876,  0.064 ],
       [ 0.9988,  0.9924,  0.9584,  0.5124,  0.1216]])

In [400]:
ssrt2.mean(axis=1)

array([[ 0.4589,  0.4669,  0.4723, ...,  0.4773,  0.5105,  0.5439],
       [ 0.4619,  0.4367,  0.4511, ...,  0.4573,  0.5209,  0.5427]])

In [399]:
ssrt1.mean(axis=1)

array([[ 0.5193,  0.4873,  0.4847, ...,  0.4987,  0.4873,  0.4669],
       [ 0.4885,  0.4919,  0.4807, ...,  0.5117,  0.4883,  0.4853]])

In [396]:
eq2

array([[ 4.80370698,  5.01370698,  5.13370698,  5.28370698,  5.51750698],
       [ 4.83590698,  5.04370698,  5.19370698,  5.36370698,  5.59370698]])

In [385]:
yhat_grouped2

array([[ 0.9832    ,  0.998     ,  0.9856    ,  0.9356    ,  0.424     ,
         0.0796    ,  5.01370698,  5.25370698,  5.46370698,  5.69370698,
         6.01370698,  4.76370698,  4.90370698,  4.99370698,  5.10370698,
         5.26370698],
       [ 0.9564    ,  0.9988    ,  0.9908    ,  0.9456    ,  0.4904    ,
         0.1208    ,  5.11890698,  5.41130698,  5.60370698,  5.82370698,
         6.14370698,  4.77370698,  4.90370698,  5.02370698,  5.12370698,
         5.29930698]])

In [383]:
yhat_grouped

array([[ 0.9808    ,  0.9992    ,  0.9896    ,  0.9344    ,  0.4056    ,
         0.07      ,  5.00990698,  5.26370698,  5.48370698,  5.68370698,
         6.02370698,  4.81370698,  5.00370698,  5.13370698,  5.28110698,
         5.50370698],
       [ 0.9568    ,  0.9996    ,  0.9944    ,  0.952     ,  0.4908    ,
         0.1196    ,  5.10370698,  5.38370698,  5.60370698,  5.82370698,
         6.13370698,  4.82370698,  5.03370698,  5.18370698,  5.32370698,
         5.55370698]])

In [412]:
#%%timeit

DVg = z + np.cumsum(np.where((rs((ncond, ntot, Tg)).T < Pg), dx, -dx).T, axis=2)
init_ss = np.array([np.array([DVc[:nss, ix] for ix in np.where(Ts<Tg, Tg-Ts, 0)]) for DVc in DVg])
s2 = init_ss[:, :, :, None] + np.cumsum(np.where(rs((ncond, nssd, nss, Ts.max()))<Ps, dx, -dx), axis=3)

In [350]:
t1 = rs((ncond, nssd, nss, Ts.max()))

In [347]:
t2 = rs((ntot/2, Ts.max()))

In [348]:
t2

array([[ 0.5488135 ,  0.71518937,  0.60276338, ...,  0.62773476,
         0.40142795,  0.41669176],
       [ 0.81083862,  0.34819194,  0.2114548 , ...,  0.56112093,
         0.83654718,  0.27805024],
       [ 0.54695006,  0.29361682,  0.96820439, ...,  0.60916405,
         0.77595844,  0.07614516],
       ..., 
       [ 0.18451403,  0.17575554,  0.64260219, ...,  0.53958294,
         0.47411735,  0.69915741],
       [ 0.66083996,  0.71201579,  0.86133357, ...,  0.64729701,
         0.41225408,  0.83713883],
       [ 0.27183393,  0.27243429,  0.63328953, ...,  0.22162773,
         0.15155548,  0.81608519]])

In [406]:
init_ss[0][:, :]

array([[ 0.15648262,  0.15648262,  0.15648262, ...,  0.15648262,
         0.15648262,  0.15648262],
       [ 0.15648262,  0.15648262,  0.15648262, ...,  0.15648262,
         0.15648262,  0.15648262],
       [ 0.15648262,  0.15648262,  0.15648262, ...,  0.15648262,
         0.15648262,  0.15648262],
       [ 0.21014825,  0.23250893,  0.22356466, ...,  0.21909253,
         0.18778757,  0.19673185],
       [ 0.24145321,  0.31300738,  0.33536806, ...,  0.28170243,
         0.25934175,  0.27275816]])

In [416]:
z

0.15424655465199996

In [407]:
s = np.cumsum(np.where(rs((ntot/2, Ts.max()))<Ps, dx, -dx), axis=1) 

In [404]:
#%%timeit

init_ss = np.array([np.array([DVc[:ntot/2, ix] for ix in np.where(Ts<Tg, Tg-Ts, 0)]) for DVc in DVg])
#DVs = init_ss[:, :, :, None] + np.cumsum(np.where(rs((ncond, nssd, ntot/2, Ts.max()))<Ps, dx, -dx), axis=3)
DSv = init_ss[:, None] + np.cumsum(np.where(rs((ntot/2, Ts.max()))<Ps, dx, -dx), axis=1) 

ValueError: operands could not be broadcast together with shapes (2,1,5,2500) (2500,900) 

In [319]:
init_ss = np.array([[DVc[:ntot/2, ix] for ix in np.where(Ts<Tg, Tg-Ts, 0)] for DVc in DVg])

In [320]:
init_ss.shape

(2, 5, 1000)

In [316]:
DVs = np.cumsum(np.where(rs((ntot/2, Ts.max()))<Ps, dx, -dx), axis=1) 

In [360]:

init_ss = np.array([np.array([DVc[:ntot/2, ix] for ix in np.where(Ts<Tg, Tg-Ts, 0)]) for DVc in DVg])
vs = init_ss[:, :, :, None] + np.cumsum(np.where(rs((ntot/2, Ts.max()))<Ps, dx, -dx), axis=1) 

In [361]:
vs.shape

(2, 5, 1000, 900)

In [317]:
DVs.shape

(1000, 900)

In [314]:
#sc = np.cumsum(np.where(rs((ntot*n, Ts.max()))<Ps, dx, -dx), axis=1)
a=np.array([1,2,3])
np.tile(a,(2,3)).shape

(2, 9)

In [275]:
%%timeit
x = rs((ncond, ntot, Tg)).T<

10 loops, best of 3: 24.2 ms per loop


In [244]:
int(ntot/len(ssd))
nss=400
nssd=5

In [253]:
rr = rs((ntot, Tg))
dvg = np.where(rr<Pg, dx, -dx)

In [257]:
Pg = 

0.67746066160103902

In [256]:
dvg.shape


(2, 2000, 346)

In [294]:
#%%timeit
DVg = z + np.cumsum(np.where((rs((ncond, ntot, Tg)).T < Pg), dx, -dx).T, axis=2)
init_ss = np.array([[DVc[:int(ntot/nssd), ix] for ix in np.where(Ts<Tg, Tg-Ts, 0)] for DVc in DVg])
DVs = init_ss[:, :, :, None] + np.cumsum(np.where(rs((ntot*ncond, Ts.max()))<Ps, dx, -dx), axis=1).reshape(ncond, nssd, int(ntot/nssd), Ts.max())

In [295]:
DVs.shape

(2, 5, 400, 900)

In [28]:
%%timeit
#collapse across SSD and get average ssrt vector for each condition
c_ssrt = ssrt.mean(axis=1)

# compute RT quantiles for correct and error resp.
gq = np.vstack([mq(rtc[rtc<tb], prob=p)*10 for rtc in grt])
eq = np.array([mq(np.extract(ert[i]<c_ssrt[i], ert[i]), prob=p)*10 for i in range(ncond)])

# Get response and stop accuracy information
gac = np.where(grt<tb, 1, 0).mean(axis=1)
sacc = np.array([1 - np.where(ert[i]<ssrt[i], 1, 0).mean(axis=1) for i in range(ncond)])

yhat = [gac, sacc, gq, eq]
yhat_grouped = np.array([np.hstack([i[ii] for i in yhat]) for ii in range(ncond)])

1000 loops, best of 3: 978 µs per loop


In [21]:
%prun iter_recost(cinits, y, nruns=5, ntrials=5000)


ERROR: Line magic function `%lprun` not found.


In [133]:
Ts

array([450, 400, 350, 301, 251])

In [170]:
%%timeit

Vpg = rs((ncond, ntot, Tg)).T
       
Vps = rs((ncond, ntot, Tg)).T

# ALL CONDITIONS, ALL SSD
DVg = z + np.cumsum(np.where((rs((ncond, ntot, Tg)).T < Pg), dx, -dx).T, axis=2)
init_ss = np.array([np.array([DVc[:nss, ix] for ix in np.where(Ts<Tg, Tg-Ts, 0)]) for DVc in DVg])
DVs = init_ss[:, :, :, None] + np.cumsum(np.where(rs((ncond, nssd, nss, Ts.max()))<Ps, dx, -dx), axis=3)

1 loops, best of 3: 277 ms per loop


In [194]:
DVg = z + np.cumsum(np.where((rs((ncond, ntot, Tg)).T < Pg), dx, -dx).T, axis=2)
init_ss = np.array([np.array([DVc[:nss, ix] for ix in np.where(Ts<Tg, Tg-Ts, 0)]) for DVc in DVg])
#DVs = init_ss[:, :, :, None] + np.cumsum(np.where(rs((ncond, nssd, nss, Ts.max()))<Ps, dx, -dx), axis=3)

In [195]:
init_ss.shape

(2, 5, 1000)

In [193]:
init_ss = np.array([[DVc[:nss, ix] for ix in np.where(Ts<Tg, Tg-Ts, 0)] for DVc in DVg])
init_ss.shape

(2, 5, 1000)

In [199]:
%%timeit
init_ss = np.array([[DVc[:int(ntot/len(ssd)), ix] for ix in np.where(Ts<Tg, Tg-Ts, 0)] for DVc in DVg])
DVs = init_ss[:, :, :, None] + np.cumsum(np.where(rs((ntot*ncond, Ts.max()))<Ps, dx, -dx), axis=1).reshape(ncond, len(ssd), int(ntot/len(ssd)), Ts.max())

10 loops, best of 3: 68.7 ms per loop


In [179]:
%%timeit
#DVs = init_ss[:, :, :, None] + np.cumsum(np.where(rs((ntot*ncond, Ts.max()))<Ps, dx, -dx), axis=1).reshape(ncond, len(ssd), int(ntot/len(ssd)), Ts.max())
DVs = np.cumsum(np.where(rs((ntot*ncond, Ts.max()))<Ps, dx, -dx), axis=1).reshape(ncond, len(ssd), int(ntot/len(ssd)), Ts.max())

10 loops, best of 3: 62.5 ms per loop


In [180]:
DVs.shape

(2, 5, 1000, 450)

In [152]:

int(ntot/len(ssd))

5

In [160]:
5*200

1000

In [161]:
np.ones_like(DVs).reshape(5, 200, 450)

array([[[ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        ..., 
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.]],

       [[ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        ..., 
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.]],

       [[ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        ..., 
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.]],

       [[ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       

In [162]:
DVs.reshape(len(ssd), int(nss/len(ssd)), Ts.max())

array([[[-0.00316228, -0.00632456, -0.00948683, ..., -0.43639432,
         -0.43955659, -0.44271887],
        [-0.00316228, -0.00632456, -0.00948683, ..., -0.42374521,
         -0.42690748, -0.43006976],
        [ 0.00316228,  0.        , -0.00316228, ..., -0.45536798,
         -0.45853026, -0.46169254],
        ..., 
        [-0.00316228, -0.00632456, -0.00948683, ..., -0.37947332,
         -0.3826356 , -0.38579787],
        [-0.00316228, -0.00632456, -0.00948683, ..., -0.51861354,
         -0.52177581, -0.51861354],
        [-0.00316228, -0.00632456, -0.00948683, ..., -0.45536798,
         -0.45853026, -0.45536798]],

       [[ 0.00316228,  0.        , -0.00316228, ..., -0.37314876,
         -0.36998649, -0.37314876],
        [-0.00316228, -0.00632456, -0.00948683, ..., -0.3541751 ,
         -0.35733738, -0.36049965],
        [ 0.00316228,  0.        ,  0.00316228, ..., -0.44904343,
         -0.45220571, -0.45536798],
        ..., 
        [-0.00316228, -0.00632456, -0.00316228, ...,

In [166]:
init_ss.shape# == 
#init_ss.reshape(1,10000)

(2, 5, 1000)

In [134]:
DVg.shape

(2, 2000, 346)

In [129]:
ixx = np.where(Ts<Tg, Tg-Ts, 0)
DVg[:, ixx.T]

array([[[ 0.15740883,  0.15740883,  0.15740883,  0.21749211,  0.28073766],
        [ 0.15740883,  0.15740883,  0.15740883,  0.19851844,  0.27441311],
        [ 0.15740883,  0.15740883,  0.15740883,  0.21116755,  0.25543944],
        ..., 
        [ 0.15740883,  0.15740883,  0.15740883,  0.19851844,  0.26808855],
        [ 0.15740883,  0.15740883,  0.15740883,  0.23646577,  0.29971133],
        [ 0.15740883,  0.15740883,  0.15740883,  0.21116755,  0.261764  ]],

       [[ 0.15108428,  0.15108428,  0.15108428,  0.19851844,  0.28706222],
        [ 0.15108428,  0.15108428,  0.15108428,  0.21749211,  0.261764  ],
        [ 0.15740883,  0.15740883,  0.15740883,  0.22381666,  0.30603588],
        ..., 
        [ 0.15740883,  0.15740883,  0.15740883,  0.17322022,  0.22381666],
        [ 0.15108428,  0.15108428,  0.15108428,  0.19851844,  0.24279033],
        [ 0.15740883,  0.15740883,  0.15740883,  0.24279033,  0.33765866]]])

In [168]:
init_ss = np.array([[DVc[:nss, ix] for ix in np.where(Ts<Tg, Tg-Ts, 0)] for DVc in DVg])


In [169]:
init_ss.shape

(2, 5, 1000)

In [111]:
init_ss[:, :, :]+DVs[:, :].T
#np.sum(init_ss, DVs., axis=2)

ValueError: operands could not be broadcast together with shapes (2,5,1000) (450,1000) 

In [84]:
%%timeit
#init_ss = np.array([np.array([DVc[:nss, ix] for ix in np.where(Ts<Tg, Tg-Ts, 0)]) for DVc in DVg])
DVs = np.cumsum(np.where(rs((nssd, nss, Ts.max()))<Ps, dx, -dx), axis=2)

10 loops, best of 3: 80 ms per loop


In [32]:
#%%timeit
# ALL CONDITIONS, ALL SSD
grt = np.where(DVg[:, nss:, :].max(axis=2)>=a, tr + np.argmax(DVg[:, nss:, :]>=a, axis=2)*dt, np.nan)
ert = np.where(DVg[:, :nss, :].max(axis=2)>=a, tr + np.argmax(DVg[:, :nss, :]>=a, axis=2)*dt, np.nan)
ssrt = np.where(np.any(DVs<=0, axis=3), ssd[:, None]+np.argmax(DVs<=0, axis=3)*dt, np.nan)

In [44]:
%%timeit
xx = tr + np.argmax(DVg[:, nss:, :]>=a, axis=2)*dt


100 loops, best of 3: 2 ms per loop


In [45]:

grt = np.where(DVg[:, nss:, :].max(axis=2)>=a, tr + np.argmax(DVg[:, nss:, :]>=a, axis=2)*dt, np.nan)


100 loops, best of 3: 3.29 ms per loop


In [47]:
grt[0][4]

nan

In [69]:
%%timeit
grt = np.where(DVg[:, nss:, :].max(axis=2)>=a, tr + np.argmax(DVg[:, nss:, :]>=a, axis=2)*dt, np.nan)
ert = np.where(DVg[:, :nss, :].max(axis=2)>=a, tr + np.argmax(DVg[:, :nss, :]>=a, axis=2)*dt, np.nan)

100 loops, best of 3: 3.5 ms per loop


In [70]:
%%timeit
yyy = np.where(DVg[:, :, :].max(axis=2)>=a, tr + np.argmax(DVg[:, :, :]>=a, axis=2)*dt, np.nan)

e = yyy[:,:nss]
g = yyy[:,nss:]

100 loops, best of 3: 3.39 ms per loop
